Для начала преобразуем данные, как предложено в условии: 
* удалим столбец 'day'
* заменим данные в столбце 'month' на индикатор летнего сезона
* добавим столбец из единиц

In [34]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as score
from sklearn.cross_validation import train_test_split
from math import exp
import numpy as nmp
import pandas

# читаем данные
data = pandas.read_csv('forestfires.csv')
# удаляем столбец 'day'
data = data.drop('day', axis=1)
# преобразуем столбец 'month'
data['month'] = data['month'].map(lambda x: 1 if x == 'jun' or x == 'jul' or x == 'aug' else 0)

# создаем массив из прочитанных данных, без поля 'area'
X = nmp.array(data.drop('area', axis=1))
# добавляем в этот массив столбец из единиц
X = nmp.hstack((X, nmp.ones((len(X), 1))))
# выделяем отдельно столбец 'area'
Y = nmp.array(data['area'])

Перейдем к обработке данных: 
* разделим все данные на основную и тестовую выборки
* обучим выборку
* применяем к тестовой выборке
* считаем среднеквдратичную ошибку

In [35]:
# делим весь массив на основную и тестовую выборку в отношении 7:3
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

# функция будет строить прямую,
# минимизируя сумму квадратов отклонения от данной прямой
clf = LR()

# обучаем выборку
clf.fit(X_train, Y_train)
# делаем "предсказание" по тестовой выборке 
prediction = clf.predict(X_test)
# считаем среднеквадратичную ошибку
accuracy = score(Y_test, prediction)

print('Accuracy without f(x):', accuracy)

Accuracy without f(x): 7887.2473386


Проведем несколько вспомогательных вычислений, чтобы получить представления о данных

In [36]:
# посчитаем количество нулевых значений в поле 'area'
sum_zeros = 0
sum_non_zeros_values = 0
# пусть будет аккуратный цикл, он смотрится симпатичнее ещё одной строки
for i in Y:
    if (i == 0.0):
        sum_zeros = sum_zeros + 1
    else:
        sum_non_zeros_values = sum_non_zeros_values + i

# всего данных - длина файла        
print('length of file:', len(data))
# количество нулей (+ процент от всего числа данных)
print('amount of zeros: ', sum_zeros, ' -> ', int(sum_zeros / len(data) * 100), '%')
# максимальное значение
print('max value: ', max(Y))
# среднее значение
print('average value: ', nmp.average(Y))
# среднее значение без учета нулей
print('average value without zeros: ', sum_non_zeros_values / (len(data) - sum_zeros))

length of file: 517
amount of zeros:  247  ->  47 %
max value:  1090.84
average value:  12.8472920696
average value without zeros:  24.6001851852


Подберем константу:
* делаем преобраование поля 'area'
* делаем все то же самое: обучаем выборку, применяем к тестовой выборке, считаем ошибку
* сравниваем получившиеся результаты для разных констант
* делаем выводы, с учетом пункта выше

In [37]:
# функция для обработки данных при разных констаантах
def try_const(c):
    print('c =', c)
    
    # делаем преобразование данных
    Y_train_transformed = nmp.log(Y_train + c)
    Y_test_transformed = nmp.log(Y_test + c)
    
    # и все, как и для непреобразованных данных
    clf.fit(X_train, Y_train_transformed)
    prediction = clf.predict(X_test)
    accuracy_transformed = score(Y_test_transformed, prediction)
    
    # вывод результатов
    print('Accuracy:', accuracy_transformed)

# выполняем функция для различных значений константы
for c in [0.1, 1, 10, 100, 1000, 10000]:
    try_const(c)

c = 0.1
Accuracy: 5.12127622752
c = 1
Accuracy: 1.80964477931
c = 10
Accuracy: 0.422218461972
c = 100
Accuracy: 0.0580179169969
c = 1000
Accuracy: 0.00378963759346
c = 10000
Accuracy: 7.14129300998e-05


Вывод:

Значения поля "area" принимают довольно маленькие значения, так что достаточно большая константа просто поглощает значение поля и ошибка, сс возрастанием константы, становится все меньше. То есть большие значения $c$ использовать не стоит. Среднее значение "area" $= 13$, так что лучше всего взять $c = 1$, так как константа ещё не имеет существнного влияние на значение, но ошибка при этом существенно меньше. 

Определим, зависит ли качество от способа разбиения выборки:

Попробуем для двух значений: $c = 0.1$ и $c = 1$

In [46]:
for i in range(1, 11):
    if (i < 6):
        c = 1
    else:
        if (i == 6):
            print()
        c = 0.1
        
    print('attempt', i, ': c = ', c) 
    #перемешиваем даннные
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)  
    # делаем преобразование данных
    Y_train_transformed = nmp.log(Y_train + c)
    Y_test_transformed = nmp.log(Y_test + c)
    
    # и все, как и для непреобрразованных данных
    clf.fit(X_train, Y_train_transformed)
    prediction = clf.predict(X_test)
    accuracy_transformed = score(Y_test_transformed, prediction)
    
    # вывод результатов
    print('Accuracy:', accuracy_transformed)

attempt 1 : c =  1
Accuracy: 2.68342721181
attempt 2 : c =  1
Accuracy: 2.08438184069
attempt 3 : c =  1
Accuracy: 2.25210367245
attempt 4 : c =  1
Accuracy: 1.70443614894
attempt 5 : c =  1
Accuracy: 1.83018421835

attempt 6 : c =  0.1
Accuracy: 9.44811217362
attempt 7 : c =  0.1
Accuracy: 9.83029099782
attempt 8 : c =  0.1
Accuracy: 5.81852638117
attempt 9 : c =  0.1
Accuracy: 6.15625643927
attempt 10 : c =  0.1
Accuracy: 5.70249083535


Вывод: значения различаются, но не существенно, в диапазоне одного порядка.